# Финальный проект по SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.

В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Описание данных**

Таблица `books`, содержит данные о книгах:

* `book_id` — идентификатор книги;
* `author_id` — идентификатор автора;
* `title` — название книги;
* `num_pages` — количество страниц;
* `publication_date` — дата публикации книги;
* `publisher_id` — идентификатор издателя.

Таблица `authors`, содержит данные об авторах:

* `author_id` — идентификатор автора;
* `author` — имя автора.

Таблица `publishers`, содержит данные об издательствах:

* `publisher_id` — идентификатор издательства;
* `publisher` — название издательства;

Таблица `ratings`, содержит данные о пользовательских оценках книг:

* `rating_id` — идентификатор оценки;
* `book_id` — идентификатор книги;
* `username` — имя пользователя, оставившего оценку;
* `rating` — оценка книги.

Таблица `reviews`, содержит данные о пользовательских обзорах на книги:

* `review_id` — идентификатор обзора;
* `book_id` — идентификатор книги;
* `username` — имя пользователя, написавшего обзор;
* `text` — текст обзора.

Задания:
- [ ] Посчитайте, сколько книг вышло после 1 января 2000 года;
- [ ] Для каждой книги посчитайте количество обзоров и среднюю оценку;
- [ ] Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- [ ] Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- [ ] Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Подключение к базе данных

In [ ]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Посчитайте, сколько книг вышло после 1 января 2000 года

In [ ]:
query_1 ='''
SELECT COUNT(book_id) as books_count
FROM books
WHERE publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(query_1, con = engine)

## Для каждой книги посчитайте количество обзоров и среднюю оценку

In [ ]:
query_2 = '''
SELECT title,
       avg_rating,
       reviews_count
FROM books
LEFT JOIN
  (SELECT book_id,
          AVG(rating) AS avg_rating
   FROM ratings
   GROUP BY book_id) AS i_ratings ON books.book_id = i_ratings.book_id
LEFT JOIN
  (SELECT book_id,
          COUNT(review_id) AS reviews_count
   FROM reviews
   GROUP BY book_id) AS i_reviews ON books.book_id = i_reviews.book_id
'''
pd.io.sql.read_sql(query_2, con = engine)

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [ ]:
query_3 ='''
SELECT i.publisher_id,
       i.publisher,
       i.books_count
FROM
  (SELECT books.publisher_id AS publisher_id,
          COUNT(books.book_id) AS books_count,
          publishers.publisher AS publisher
   FROM books
   LEFT JOIN publishers ON books.publisher_id=publishers.publisher_id
   WHERE num_pages>50
   GROUP BY books.publisher_id,
            publishers.publisher
   ORDER BY books_count DESC) AS i
LIMIT 1;
'''
pd.io.sql.read_sql(query_3, con = engine)

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [ ]:
query_4 = '''
SELECT author_avg.author,
       AVG(author_avg.avg_rating) AS average_rating
FROM
  (SELECT authors.author AS author ,
          AVG(ratings.rating) AS avg_rating
   FROM books
   LEFT JOIN authors ON books.author_id = authors.author_id
   LEFT JOIN ratings ON books.book_id = ratings.book_id
   GROUP BY books.book_id ,
            authors.author_id
   HAVING COUNT(ratings.rating_id) >=50) AS author_avg
GROUP BY author_avg.author
ORDER BY average_rating DESC
LIMIT 1
'''
pd.io.sql.read_sql(query_4, con = engine)

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [ ]:
query_5 = '''
SELECT AVG(i.reviews_count)
FROM
  (SELECT username,
          COUNT(review_id) AS reviews_count
   FROM reviews
   WHERE username IN
       (SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(rating_id) > 50)
   GROUP BY username) AS i;
'''
pd.io.sql.read_sql(query_5, con = engine)

## Общий вывод

Решив поставленные задачи, выяснили следующее:

1) После 1 января 2000 года вышло **819 книг**;

2) Для представленных 1000 книг рассчитали среднюю оценку и количество обзоров;

3) Издательство, которое выпустило наибольшее число книг толще 50 страниц - это **Penguin Books, выпустившее 42 книги**;

4) Автор с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками - это **J.K. Rowling/Mary GrandPré со средней оценкой 4.28**;

5) Среднее количество обзоров от пользователей, которые поставили больше 50 оценок: **24.33**.